In [ ]:
import pl.mareklangiewicz.kommand.*
import pl.mareklangiewicz.kommand.CliPlatform.Companion.SYS
import pl.mareklangiewicz.kommand.core.*
import pl.mareklangiewicz.kommand.core.LsOpt.*
import pl.mareklangiewicz.kommand.gnome.*
import pl.mareklangiewicz.kommand.gnome.GnomeExt.Cmd.list

SYS.run {
    start(gnometerm(bash(gnomeext(list), pause = true)))
    start(gnometerm(bash(ls { -LongFormat; -All }, pause = true)))
}




In [ ]:
SYS.run {
    LsSamples.lsParentWithSlashes.exec().printlns()
}

In [ ]:
import pl.mareklangiewicz.kommand.konfig.*

SYS.konfigInUserHomeConfigDir().printAll()

In [ ]:
import pl.mareklangiewicz.kommand.admin.*
import pl.mareklangiewicz.kommand.debian.*
import pl.mareklangiewicz.kommand.debian.DpkgAct.*
import pl.mareklangiewicz.kommand.samples.Samples

SYS.run {
    val kmd =
//        Samples.GitHub.help
//        Samples.GitHub.secretSetFakeSecretInAbcdK
//        Samples.GitHub.secretListForAbcdK
//        dpkg { -DpkgAct.PrintArch }
//        dpkg(Status("apt"))
//        dpkg(Details("findutils"))
//        sudo(bash("whoami; pwd; ls"))
//        sudo(kommand("aptitude"))
//        sudoEdit("/etc/hosts")
        sudoEdit("/etc/fstab")
//        kommand("whoami").withSudo(SudoOpt.User(zenityAskForEntryExec("Enter some user name")))
    startInGnomeTermIfUserConfirms(kmd)
}


In [ ]:
import pl.mareklangiewicz.kommand.debian.*

SYS.dpkgSearchOneCommandExec("find")

In [ ]:
import pl.mareklangiewicz.kommand.admin.*
import pl.mareklangiewicz.kommand.find.*

SYS.run {
//    sudoExec(ls("/root/", withHidden = true), inPass = zenityAskForPasswordExec())
//    FindSamples.findAbcIgnoreCase.exec()
//    FindSamples.findSomeSamples.exec()
//    FindSamples.findNodeModulesDirs.exec().printlns()
//    FindSamples.findBuildDirs.exec().printlns()
//    FindSamples.findSymLinksToKtsFilesInDepsKt.exec().printlns()
    FindSamples.findDepthMax2FilesInDepsKtAndRunFileOnEach.exec().printlns()
//    FindSamples.findInKotlinKtFilesModifiedIn24h.exec().printlns()
}

In [ ]:
import pl.mareklangiewicz.kommand.find.*

SYS.findTypicalDetailsTableExec("..")


In [ ]:
import pl.mareklangiewicz.kommand.find.*

SYS.findMyKotlinCodeExec(withModifTime24h = FindExpr.NumArg.LessThan(8)).printlns()